In [1]:
import KeyFunctions as kf

root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

In [2]:
#Import the postcode list from the raw data gdf
gdf = kf.load_obj(root_path,"raw_gdf")
# name = "StPauls" 
# gdf = kf.load_obj(root_path,"raw_gdf_" + name)

#2m29s

In [3]:
#gdf["Type"].unique().tolist()

['RailwayStations',
 'FerryTerminals',
 'BusStops',
 'RoadAccidents',
 'Postcodes',
 'LReg',
 'FireStations',
 'Schools',
 'Crimes',
 'StopAndSearch',
 'Clinic',
 'Dentists',
 'GP',
 'Hospital',
 'Opticians',
 'Pharmacy',
 'Care homes and care at home',
 'All_GB',
 'LocalAuthorities',
 'LSOA',
 'GreenSpace',
 'Rivers',
 'Railway_Lines',
 'Urban_Region',
 'Coastline',
 'Lakes',
 'Road',
 'LocalNatureReserve',
 'ConservationArea',
 'Listed Buildings',
 'MetOffice']

In [4]:
#Initialise our postcodes gdf as a list of postcodes and their points
Postcodes = gdf.loc[gdf["Type"]=="Postcodes",["Name", "geometry"]].copy()
Postcodes = Postcodes.rename(columns={"Name": "Postcode"})

Postcodes = Postcodes.set_index("Postcode")

In [5]:
#List out the points we will want to use for densities, point distances...etc.

All_Crimes = gdf.loc[gdf["Type"]=="Crimes",:]
All_Schools = gdf.loc[gdf["Type"]=="Schools",:]
Hospitals = gdf.loc[gdf["Type"]=="Hospital",:]
RailwayStations = gdf.loc[gdf["Type"]=="RailwayStations",:]
ListedBuildings = gdf.loc[gdf["Type"]=="Listed Buildings",:]
PropertySales = gdf.loc[gdf["Type"]=="LReg",:]

TotalRainfall_mm_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='TotalRainfall_mm_2020'),:]
Snow_Days_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='Snow_Days_2020'),:]
Sunlight_h_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='Sunlight_h_2020'),:]
AverageTemperature_C_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='AverageTemperature_C_2020'),:]
GroundFrost_Days_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='GroundFrost_Days_2020'),:]
#...etc.

#Point datasets to find the density of and the radius of those densities in the format output_name:dataframe where the output variable will have the name Outname_Density_Radius
density_var_dict = {
    "Postcodes":Postcodes, 
    "All_Crimes":All_Crimes, 
    "All_Schools":All_Schools, 
    "Hospitals":Hospitals, 
    "RailwayStations":RailwayStations,
    "ListedBuildings":ListedBuildings
}

density_r_list = [1000,5000,25000]

#Point datasets to find the nearest of in the format output_name:dataframe where the output variable will have the name Nearest_Out_Name
nearest_var_dict = {
    "School":All_Schools, 
    "Hospital":Hospitals, 
    "RailwayStation":RailwayStations,
    "ListedBuilding":ListedBuildings
}

#Points to find the average of
average_var_dict = {
    "ProertyPrice_2020_2021":[PropertySales, "Details_Float"],
    "TotalRainfall_mm_2020":[TotalRainfall_mm_2020, "Details_Float"],
    "Snow_Days_2020":[Snow_Days_2020, "Details_Float"],
    "Sunlight_h_2020":[Sunlight_h_2020, "Details_Float"],
    "AverageTemperature_C_2020":[AverageTemperature_C_2020, "Details_Float"],
    "GroundFrost_Days_2020":[GroundFrost_Days_2020, "Details_Float"]
}

In [14]:
#Start with adding density columns
for k in density_var_dict:
    for r in density_r_list:
        Postcodes[k + "_Density_" + str(r)] = kf.density_within_radius(Postcodes,density_var_dict[k], r)

#28m21s

In [15]:
#Add distance to the nearest, don't include geometry lines
keep_n_g_d = [False, False, True]
for k in nearest_var_dict:
    Postcodes = kf.nearest_neighbor_point(Postcodes, nearest_var_dict[k], "Name", k, keep_n_g_d = keep_n_g_d, merge=True)

#3m24s

In [6]:
#Find the average value within the list of radii
for k in average_var_dict:
    for r in density_r_list:
        Postcodes["Average_" + k + "_" + str(r)] = kf.average_within_radius(Postcodes, average_var_dict[k][0], average_var_dict[k][1], r)

In [7]:
Postcodes.columns

Index(['geometry', 'Average_ProertyPrice_2020_2021_1000',
       'Average_ProertyPrice_2020_2021_5000',
       'Average_ProertyPrice_2020_2021_25000',
       'Average_TotalRainfall_mm_2020_1000',
       'Average_TotalRainfall_mm_2020_5000',
       'Average_TotalRainfall_mm_2020_25000', 'Average_Snow_Days_2020_1000',
       'Average_Snow_Days_2020_5000', 'Average_Snow_Days_2020_25000',
       'Average_Sunlight_h_2020_1000', 'Average_Sunlight_h_2020_5000',
       'Average_Sunlight_h_2020_25000',
       'Average_AverageTemperature_C_2020_1000',
       'Average_AverageTemperature_C_2020_5000',
       'Average_AverageTemperature_C_2020_25000',
       'Average_GroundFrost_Days_2020_1000',
       'Average_GroundFrost_Days_2020_5000',
       'Average_GroundFrost_Days_2020_25000'],
      dtype='object')

In [8]:
Postcodes.head()

,geometry,Average_ProertyPrice_2020_2021_1000,Average_ProertyPrice_2020_2021_5000,Average_ProertyPrice_2020_2021_25000,Average_TotalRainfall_mm_2020_1000,Average_TotalRainfall_mm_2020_5000,Average_TotalRainfall_mm_2020_25000,Average_Snow_Days_2020_1000,Average_Snow_Days_2020_5000,Average_Snow_Days_2020_25000,Average_Sunlight_h_2020_1000,Average_Sunlight_h_2020_5000,Average_Sunlight_h_2020_25000,Average_AverageTemperature_C_2020_1000,Average_AverageTemperature_C_2020_5000,Average_AverageTemperature_C_2020_25000,Average_GroundFrost_Days_2020_1000,Average_GroundFrost_Days_2020_5000,Average_GroundFrost_Days_2020_25000
Postcode,,,,,,,,,,,,,,,,,,,
CM233WE,POINT (530754.000 182754.000),2.693186e+06,2.340041e+06,2.317996e+06,698.104965,683.134901,683.134901,0.030513,0.025311,0.025311,1730.156347,1732.923776,1732.923776,12.787322,12.794034,12.794034,58.065582,55.770707,55.770707
E11AB,POINT (534253.000 181031.000),1.170241e+06,2.327521e+06,2.317996e+06,675.888767,683.134901,683.134901,0.024098,0.025311,0.025311,1735.767290,1732.923776,1732.923776,12.793385,12.794034,12.794034,53.673690,55.770707,55.770707
E11AE,POINT (534279.000 181467.000),1.239140e+06,2.338053e+06,2.317996e+06,679.804446,683.134901,683.134901,0.028327,0.025311,0.025311,1736.629681,1732.923776,1732.923776,12.769558,12.794034,12.794034,53.609399,55.770707,55.770707
E11AF,POINT (534253.000 181031.000),1.170241e+06,2.327521e+06,2.317996e+06,675.888767,683.134901,683.134901,0.024098,0.025311,0.025311,1735.767290,1732.923776,1732.923776,12.793385,12.794034,12.794034,53.673690,55.770707,55.770707
E11AG,POINT (534291.000 181451.000),1.192058e+06,2.338053e+06,2.317996e+06,679.804446,683.134901,683.134901,0.028327,0.025311,0.025311,1736.629681,1732.923776,1732.923776,12.769558,12.794034,12.794034,53.609399,55.770707,55.770707
